In [1]:
ratingsFile = open('pruned_data.csv',encoding = 'utf8',errors='ignore')
users = {}
business = {}
user_counter = 0
business_counter = 0
row = []
col = []
data = []
for line in ratingsFile:
    lineValues = line.split(',')
    user_id = lineValues[0]
    business_id = lineValues[1]
    rating = float(lineValues[2])
    data.append(rating)
    if user_id not in users:
        row.append(user_counter)
        users[user_id] = user_counter
        user_counter += 1
    else:
        row_number = users[user_id]
        row.append(row_number)
    if business_id not in business:
        col.append(business_counter)
        business[business_id] = business_counter
        business_counter += 1
    else:
        col_number = business[business_id]
        col.append(col_number)
ratingsFile.close()

In [2]:
import scipy.sparse as sp_sparse
import numpy as np
row = np.array(row)
col = np.array(col)
print(col)
print(row)
data = np.array(data)
M = sp_sparse.csr_matrix((data,(row,col)),shape=(len(users),len(business)))
M = M.T
print(M)
print(M.get_shape())
#print(M.toarray())

[   0    1    2 ...  767 1189 3023]
[   0    0    0 ... 4793 4793 4793]
  (0, 0)	4.0
  (1, 0)	4.0
  (2, 0)	5.0
  (3, 0)	4.0
  (4, 0)	4.0
  (5, 0)	5.0
  (6, 0)	4.0
  (7, 0)	4.0
  (8, 0)	2.0
  (9, 0)	5.0
  (10, 0)	5.0
  (11, 0)	4.0
  (12, 0)	5.0
  (13, 0)	4.0
  (14, 0)	4.0
  (15, 0)	4.0
  (16, 0)	2.0
  (17, 0)	3.0
  (18, 0)	4.0
  (19, 0)	4.0
  (20, 0)	5.0
  (21, 0)	3.0
  (22, 0)	4.0
  (23, 0)	4.0
  (24, 0)	4.0
  :	:
  (2322, 4792)	4.0
  (2423, 4792)	5.0
  (2741, 4792)	4.0
  (2845, 4792)	2.0
  (2982, 4792)	5.0
  (3138, 4792)	3.0
  (40, 4793)	5.0
  (63, 4793)	5.0
  (414, 4793)	3.0
  (767, 4793)	5.0
  (814, 4793)	4.0
  (916, 4793)	3.0
  (972, 4793)	4.0
  (1076, 4793)	5.0
  (1189, 4793)	1.0
  (1634, 4793)	3.0
  (1928, 4793)	2.0
  (2218, 4793)	5.0
  (2388, 4793)	4.0
  (2593, 4793)	5.0
  (2995, 4793)	3.0
  (3023, 4793)	5.0
  (3068, 4793)	5.0
  (3429, 4793)	5.0
  (3893, 4793)	3.0
(4092, 4794)


In [3]:
from scipy.sparse import csr_matrix
import random
#print(M.get_shape)
non_zero_cells = csr_matrix.nonzero(M)
# print(non_zero_cells[0])
removed_cells = []
num_non_zero = len(non_zero_cells[0])
cells_to_remove = int(0.05*len(non_zero_cells[0]))
while cells_to_remove > 0:
    k = random.randint(0,num_non_zero-1)
    i = non_zero_cells[0][k]
    j = non_zero_cells[1][k]
    if (i,j) not in removed_cells:
        M[i,j] = 0
        removed_cells.append((non_zero_cells[0][i],non_zero_cells[1][i]))
        cells_to_remove -= 1
#print(cells_to_remove)
#print(non_zero_cells)
#print(removed_cells)

* Εδώ υπολογίζω τα similaritie επιχείρησης με επιχείρηση. Συνεπώς ο πίνακας αυτός θα είναι τετραγωνικός Διάστασης (Επιχειρήσεις Χ Επιχειρήσεις) = (4092,4092)

In [4]:
import sklearn.metrics as metrics
M_cosine_similarity = metrics.pairwise.cosine_similarity(M)
#M_cosine_similarity = csr_matrix(M_cosine_similarity)
print(len(business))
print(M_cosine_similarity.shape)

4092
(4092, 4092)


In [5]:
import sys
import time
import copy
import numpy as np
from scipy.sparse import csc_matrix,lil_matrix
M_csc = csr_matrix.tocsc(M)
np.fill_diagonal(M_cosine_similarity,0)
predictions = []
cell_counter = 0
start_time = time.time()
for removed_cell in removed_cells:

    #topk = []
    #Με j είναι οι χρήστρες και i είναι οι επιχειρήσεις
    i = removed_cell[0]
    j = removed_cell[1]
    #αριθμός όμοιων επιχειρήσεων που θα συμβάλουν στον υπολογισμό πρόβλεψης της βαθμολογίας που θα έβαζε ο j χρήστης
    k=100

    #Εξάγω τις θέσεις των κελιών όπου ο j χρήστης έχεις δώσει βαθμολογία για κάποια επιχείρηση
    non_zero_ratings = csc_matrix.nonzero(M_csc[:,j])

    number = len(non_zero_ratings[0])

    #Αυτή η μάσκα έχει άσους στις θέσεις όπου τα ratings για την επιχείρηση είναι μη μηδενικά
    mask_data = np.ones((number))
    mask_row = np.zeros((number),dtype=int)
    mask_col = non_zero_ratings[0]
    mask = sp_sparse.csr_matrix((mask_data,(mask_row,mask_col)),shape=(1,len(M_cosine_similarity[i,:])))
    
    #Εδώ κρατάμε ένα slice με το similarity της επιχείρησης που μας ενδιαφέρει
    similarity_row = M_cosine_similarity[i,:]
    similarity_row = sp_sparse.csr_matrix(similarity_row)

    similarity_row = similarity_row.toarray()
    mask = mask.toarray()
    
    #Υπολογίζω το vector το οποίο θα έχει τα similarities που με ενδιαφέρουν, δηλαδή των επιχειρήσεων που έχει δώσει βαθμολογία ο χρήστης
    result = similarity_row * mask

    #φτιάχνω μια νέα μάσκα στην οποία πια θα κρατήσω μόνο τις K μεγαλύτερες όμοιότητες
    new_mask = lil_matrix((1,len(M_cosine_similarity[i,:])), dtype=float)
    #φτιάχνω ένα αντίγραφο του πίνακα με τα συνολικά similarities για να έχω πρόσβαση στις πληροφορίες του πίνακα μετά την τροποποίηση του
    #λόγω της διαδικασίας αναζήτησης των Κ κορυφαίων ομοιοτήτων μέσα στον πίνακα
    copied = copy.deepcopy(result)
    
    #result = lil_matrix(result)nnew_maskew_mask
    #Περιορίζω τον αριθμό των Κ κορυφαίων ομοιοτήτων που θα συμβάλουν στον υπολογισμό ανάλογα με πόσες είναι μη μηδενικές
    #ίσως και να μην χρειάζεται αλλά ίσως κερδίσω κάποιο χρόνο στους υπολογισμούς, βασικά θα γίνει πιο γρήγορα η εξαγωγή των κορυφαίων ομοιοτήτων
    #μιας και το να αντλώ μηδενικές ομοιότητες οι οποίες δεν θα συμβάλουν στο αποτέλεσμα δεν έχει νόημα
    number_of_non_zero_similarities = np.nonzero(result)
    if k > number_of_non_zero_similarities:
        k = number_of_non_zero_similarities
    
    while k > 0 and number_of_non_zero_similarities < : #and len(np.nonzero(result))>k 
        index = np.argmax(result)
        # print(index)
        #topk.append((index,result[0][index]))
        
        #τοποθετώ το max similarity στο νέο κενό πίνακα που θα χρησιμοποιήσω για τον υπολογισμό της πρόβλεψης της βαθμολογίας
        new_mask[0,index] = copied.item(0,index)
        
        #θέτω 0 το max similarity που βρήκα για να πάρω με το argmax την θέση του αμέσως μεγαλύτερου από αυτό
        result[0][index] = 0
        k -= 1
        
    sum_of_sim = 0
    sum_of_rating = 0
    #new_mask = new_mask.toarray()
    
    #υπολογίζω το άθροισμα του παρονομαστή
    sum_of_sim = np.sum(new_mask)
    #sum_of_sim = new_mask.sum()
    
    #column = sp_sparse.lil_matrix(M[:,j])
    #υπολογίζω το άθροισμα του αριθμητή
    sum_of_rating = new_mask * M[:,j]
    sum_of_rating = np.sum(sum_of_rating)
    cell_counter += 1

    # for element in topk:
    #     sum_of_sim += element[1]
    #     #print(M[element[0],j])
    #     sum_of_rating += M[element[0],j] * element[1]
    # if sum_of_rating == sumsim:
    #     cell_counter += 1
    if sum_of_sim != 0:
        prediction = sum_of_rating/sum_of_sim
    else:
        prediction = 0
    predictions.append((i,j,prediction,sum_of_rating))
    #καταχωρώ την πρόβλεψη στον πίνακα με τα reviews
    M[i,j] = prediction
    
end_time = time.time()
total_time = end_time -start_time
print(total_time)
print(cell_counter)

(array([   0,   53,   62,  116,  170,  189,  230,  254,  288,  293,  340,
        386,  393,  445,  456,  499,  569,  646,  665,  683,  729,  793,
        842, 1010, 1029, 1057, 1058, 1093, 1227, 1296, 1351, 1427, 1786,
       2078, 2097, 2182, 2222, 2303, 2435, 2465, 2567, 2764, 3040, 3130,
       3208, 3547, 3686, 3720], dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int32))


SystemExit: 1

/home/steve/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print(len(predictions))
print(len(removed_cells))

In [ ]:
print(predictions)